In [ ]:
!pip install outlines transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.7/912.7 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 1.8 MB/s eta 0:00:00
   

This code generates synthetic in-domain data, leveraging on the terminologies given in the dev set. This is done only for the track 1 since the dev data is not released for track 2.

Checks needed:

1. Check if the target language is an actual target language
2. Check the quality of synthetic data:
- use cross entropy difference between data and test set?




In [ ]:
import torch
from huggingface_hub import login
login(token='your huggingface token')
from outlines.models.transformers import Transformers
import outlines
from pydantic import BaseModel, Field, validator
from typing import List, Literal, TypedDict
from enum import Enum
from outlines.samplers import multinomial


model_name = "CohereLabs/aya-expanse-8b"
#tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = AutoModelForCausalLM.from_pretrained(model_id)
model = outlines.models.transformers(
    model_name,
    device="cuda" if torch.cuda.is_available() else "cpu",
    model_kwargs={"device_map": "auto"} # Add this line
)

/tmp/ipython-input-2-2560743419.py:15: DeprecationWarning: 
        The `transformers` function is deprecated starting from v1.0.0.
        Do not use it. Support for it will be removed in v1.1.0.
        Instead, you should instantiate a `Transformers` model with the
        `outlines.from_transformers` function that takes a transformers model
        and a tokenizer as arguments. For example:
        ```python
        from transformers import AutoModelForCausalLM, AutoTokenizer
        from outlines import from_transformers
        model = from_transformers(
            AutoModelForCausalLM.from_pretrained("<model_name>"),
            AutoTokenizer.from_pretrained("<model_name>")
        )
        ```
        
  model = outlines.models.transformers(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://h

config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/8.64k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

In [ ]:
### Load dev data ####
import json
from google.colab import drive
drive.mount('/content/drive')

dev_repo = "/content/drive/MyDrive/dev/enes_dev.jsonl"

dev = []
with open(dev_repo, "r") as f:
  for line in f:
    dev.append(json.loads(line))



Mounted at /content/drive


In [ ]:
print(dev[0])

{'en': 'This service describes the deployed (run-time) state of SAP HANA database artifacts, for example: tables, views, or procedures, which have been created or adjusted by the SAP Integrated Development Environment (WebIDE) editors as a family of consistent design-time artifacts for all key SAP HANA platform database features.\n', 'de': 'Dieser Service beschreibt den implementierten Zustand (Laufzeitzustand) von SAP-HANA-Datenbankartefakten, z.\xa0B. Tabellen, Views oder Prozeduren, die von den SAP-Integrated-Development-Environment-Editoren (WebIDE-Editoren) als eine Familie konsistenter Entwurfszeit-Artefakte für alle wichtigen SAP-HANA-Plattform-Datenbankfunktionen erstellt oder angepasst wurden.\n', 'proper_terms': {'design': 'Entwurf', 'state': 'Zustand'}, 'random_terms': {'artifacts': 'Artefakten', 'key': 'wichtigen'}}


In [ ]:
def generate_synth_data(dev, number_of_sents, source_language, target_language):
  all_synth_data = []
  for sample in dev:
    terms = ', '.join(sample['proper_terms'].keys())
    messages = [{"role": "user", "content": f"Please use the terms '{terms}' to generate {number_of_sents} sentences in {source_language}-{target_language} about information technology."}]
    #messages = [{"role": "user", "content": f"Please use the terms \"{sample["proper_terms"].keys()}\" to generate {number_of_sents} sentences in {source_language}-{target_language} whose content is related to technology."}]
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    ## <BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Anneme onu ne kadar sevdiğimi anlatan bir mektup yaz<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>
    gen_tokens = model.generate(
        input_ids,
        max_new_tokens=100,
        top_p=1.0,
        do_sample=True,
        temperature=0.00001,
        )
    gen_text = tokenizer.decode(gen_tokens[0])

    print(gen_text)
    all_synth_data.append(gen_text)
  return all_synth_data



In [ ]:
def constrained_inference(dev, number_of_sents, source_language, target_language):
  all_synth_data = []
  count = 0
  for sample in dev:
    print(f'terms to include: {", ".join(list(sample["proper_terms"].keys()))}')
    prompt =  f"""Please use the terms \"{", ".join(list(sample["proper_terms"].keys()))}\" to generate {number_of_sents} full sentences in {source_language}-{target_language} whose content is related to technology. Return a JSON dictionary where the key is the numbered integer, and each value is a dictionary with keys 'en' for English and 'es' for Spanish."""
   #sampler=GreedySampler() #for some reason the GreedySampler() does not work as it sets temperature to 0, But I am also using huggingface transformers which requires a temperature parameter
    sampler = multinomial(temperature=0.00001, top_p=1.0)
    structured_model = outlines.generate.json(model, NumberedSentences, sampler)
    result = structured_model(prompt)
    clean_result = {k: v.translations for k, v in result.sentences.items()}  # Extract the inner dict
    print("clean", clean_result)
    all_synth_data.append(clean_result)
    count+=1
    print(count)

  return all_synth_data

In [ ]:
from pydantic import validator
## constrained decoding ##
from typing import Dict

class SentencePair(BaseModel):
    translations: Dict[Literal["en", "es"], str] = Field(
        ...,
        description="Dictionary containing English ('en') and Spanish ('es') translations"
    )
    #validator to ensure that the keys of the translation pairs are 'en' and 'de'
    @validator('translations', pre=True)
    def convert_lang_names(cls, v):
        mapping = {"English": "en", "Spanish": "es"}
        return {mapping.get(k, k): v for k, v in v.items()}


class NumberedSentences(BaseModel):
    sentences: Dict[str, SentencePair] = Field(
        ...,
        description="Dictionary of numbered sentence pairs that contains the particular source term"
    )
    '''
    #to ensure that the keys are integers
    @validator('sentences', pre=True)
    def convert_keys_to_int(cls, v):
        return {int(k.strip('.')) if isinstance(k, str) else k: v for k, v in v.items()}
    '''



/tmp/ipython-input-6-4109222890.py:11: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('translations', pre=True)


In [ ]:
if __name__ == "__main__":
  output_repo = "/content/drive/MyDrive/wmt2025/enes_syn"
  all_synth_data =constrained_inference(dev, 20, "English", "Spanish")
  with open(output_repo, "w") as f:
    json.dump(all_synth_data, f)


terms to include: item


/tmp/ipython-input-5-1889207284.py:9: DeprecationWarning: 
        The `json` function is deprecated starting from v1.0.0.
        Do not use it. Support for it will be removed in v1.1.0.
        Use the `Generator` object instead:
        ```python
        from outlines import Generator
        from outlines.types import JsonSchema
        schema_str = '...'  # JSON Schema as a string
        generator = Generator(model, JsonSchema(schema_str))
        ```
        You can then call the generator created with a prompt to generate
        JSON data that matches the schema.
        
  structured_model = outlines.generate.json(model, NumberedSentences, sampler)


clean {'1': {'en': 'The item is a new technology.', 'es': 'El artículo es una nueva tecnología.'}, '2': {'en': 'This item is a technological innovation.', 'es': 'Este artículo es una innovación tecnológica.'}, '3': {'en': 'The item is a cutting-edge technology.', 'es': 'El artículo es una tecnología de vanguardia.'}, '4': {'en': 'The item is a digital technology.', 'es': 'El artículo es una tecnología digital.'}, '5': {'en': 'This item is a revolutionary technology.', 'es': 'Este artículo es una tecnología revolucionaria.'}, '6': {'en': 'The item is a smart technology.', 'es': 'El artículo es una tecnología inteligente.'}, '7': {'en': 'The item is a high-tech technology.', 'es': 'El artículo es una tecnología de alta tecnología.'}, '8': {'en': 'This item is a futuristic technology.', 'es': 'Este artículo es una tecnología futurista.'}, '9': {'en': 'The item is a green technology.', 'es': 'El artículo es una tecnología verde.'}, '10': {'en': 'The item is a clean technology.', 'es': 'El 

In [ ]:
  with open(output_repo, "w") as f:
    json.dump(all_synth_data, f)

NameError: name 'all_synth_data' is not defined